In [37]:
import numpy as np 
import pandas as pd 
from river.datasets import synth
from river.datasets import AirlinePassengers, Elec2

# Task 1 - Hyper-parameter tuning 

In [28]:
datastream1 = synth.RandomRBFDrift(seed_model=42, seed_sample=42, n_classes=2, n_features=10, n_centroids=20, n_drift_centroids=10)
datastream2 = synth.LEDDrift(seed=42, noise_percentage=0.2, n_drift_features=5)
datastream3 = AirlinePassengers() # Only 144 items 
datastream4 = Elec2()

In [81]:
# Create and store the datastreams 

data1 = datastream1.take(5000)
df1 = pd.DataFrame([{**x, "label": y} for x, y in data1])

data2 = datastream2.take(5000)
df2 = pd.DataFrame([{**x, "label": y} for x, y in data2])

data3 = datastream3.take(k=144)
df3 = pd.DataFrame([{**x, "label": y} for x, y in data3])

data4 = datastream4.take(5000)
if datastream4.is_downloaded: # Check if the csv with the datastream is downloaded 
    df4 = pd.read_csv(datastream4.path) # Load the csv file into a dataframe 
else:
    df1 = pd.DataFrame([{**x, "label": y} for x, y in data4])

df1.to_csv("./Datastreams/datastream1.csv")
df2.to_csv("./Datastreams/datastream2.csv")
df3.to_csv("./Datastreams/datastream3.csv")
df4.to_csv("./Datastreams/datastream4.csv")